In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import torch

In [50]:
import torch
import os

class CFG:
    def __init__(self, mode="train", kaggle_notebook=False, debug=False):
        assert mode in ["train", "inference"], "mode must be 'train' or 'inference'"
        self.mode = mode
        self.KAGGLE_NOTEBOOK = kaggle_notebook
        self.debug = debug

        # ===== Path Settings =====
        if self.KAGGLE_NOTEBOOK:
            self.OUTPUT_DIR = ''
            self.train_datadir = '/kaggle/input/birdclef-2025/train_audio'
            self.train_csv = '/kaggle/input/birdclef-2025/train.csv'
            self.test_soundscapes = '/kaggle/input/birdclef-2025/test_soundscapes'
            self.submission_csv = '/kaggle/input/birdclef-2025/sample_submission.csv'
            self.taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv'
            self.spectrogram_npy = '/kaggle/input/birdclef25-mel-spectrograms/birdclef2025_melspec_5sec_256_256.npy'
            self.model_path = '/kaggle/input/birdclef-2025-0330'
        else:
            self.OUTPUT_DIR = '../data/result/'
            self.PROCESSED_DIR = '../data/processed/'
            self.train_datadir = '../data/raw/train_audio/'
            self.train_csv = '../data/raw/train.csv'
            self.test_soundscapes = '../data/raw/test_soundscapes/'
            self.submission_csv = '../data/raw/sample_submission.csv'
            self.taxonomy_csv = '../data/raw/taxonomy.csv'
            self.spectrogram_npy = '../data/processed/mel-spec_0329/birdclef2025_melspec_5sec_256_256.npy'
            self.models_dir = "../models/" # 全modelの保存先
            self.model_path = self.models_dir # 各モデルの保存先．学習時に動的に変更．

        # ===== Model Settings =====
        self.model_name = 'efficientnet_b0'
        self.pretrained = True if mode == "train" else False
        self.in_channels = 1

        # ===== Audio Settings =====
        self.FS = 32000
        self.WINDOW_SIZE = 5.0
        self.TARGET_DURATION = 5.0
        self.TARGET_SHAPE = (256, 256)
        self.N_FFT = 1024
        self.HOP_LENGTH = 512
        self.N_MELS = 128
        self.FMIN = 50
        self.FMAX = 14000        

        # ===== Training Mode =====
        if mode == "train":
            self.seed = 42
            self.apex = False
            self.print_freq = 100
            self.num_workers = 2

            self.LOAD_DATA = True
            self.epochs = 10
            self.batch_size = 32
            self.criterion = 'BCEWithLogitsLoss'

            self.n_fold = 5
            self.selected_folds = [0, 1, 2, 3, 4]

            self.optimizer = 'AdamW'
            self.lr = 5e-4
            self.weight_decay = 1e-5
            self.scheduler = 'CosineAnnealingLR'
            self.min_lr = 1e-6
            self.T_max = self.epochs

            self.aug_prob = 0.5
            self.mixup_alpha = 0.5
            
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

            if self.debug:
                self.epochs = 2
                self.selected_folds = [0]
                self.batch_size = 4
                
        # ===== Inference Mode =====
        elif mode == "inference":
            self.batch_size = 16
            self.use_tta = False
            self.tta_count = 3
            self.threshold = 0.5

            self.use_specific_folds = False
            self.folds = [0, 1]  # Used only if use_specific_folds is True

            self.debug_count = 3
            
            self.device = "cpu"
cfg = CFG()

In [51]:
train_label = pd.read_csv(cfg.train_csv)
submission = pd.read_csv(cfg.submission_csv)
taxonomy = pd.read_csv(cfg.taxonomy_csv)

In [52]:
train_label.head()

# secoundary labels
train_label[train_label["secondary_labels"] != "['']"]

# 鳴き声タイプ
train_label[train_label["type"] != "['']"].head()

,primary_label,secondary_labels,type,filename,collection,rating,url,latitude,longitude,scientific_name,common_name,author,license
9,126247,"['65448', '22976', '476538']",['advertisement call'],126247/XC941297.ogg,XC,3.5,https://xeno-canto.org/941297,9.0465,-79.3024,Leptodactylus insularum,Spotted Foam-nest Frog,Chris Harrison,cc-by-nc-sa 4.0
20,134933,[''],['advertisement call'],134933/XC941298.ogg,XC,4.0,https://xeno-canto.org/941298,8.6260,-80.1392,Espadarana prosoblepon,Emerald Glass Frog,Chris Harrison,cc-by-nc-sa 4.0
130,22333,[''],['advertisement call'],22333/XC890507.ogg,XC,5.0,https://xeno-canto.org/890507,12.1205,-68.9658,Eleutherodactylus johnstonei,Lesser Antillean whistling frog,Chris Harrison,cc-by-nc-sa 4.0
131,22333,[''],['territorial call'],22333/XC894982.ogg,XC,4.0,https://xeno-canto.org/894982,12.1187,-68.2912,Eleutherodactylus johnstonei,Lesser Antillean whistling frog,Baudewijn Odé,cc-by-nc-sa 4.0
177,22973,[''],['flight call'],22973/XC167037.ogg,XC,4.0,https://xeno-canto.org/167037,3.8284,-67.9020,Leptodactylus fuscus,Whistling Grass Frog,Ottavio Janni,cc-by-nc-sa 3.0


In [53]:
train_label[train_label["rating"] != 0.0]

,primary_label,secondary_labels,type,filename,collection,rating,url,latitude,longitude,scientific_name,common_name,author,license
9,126247,"['65448', '22976', '476538']",['advertisement call'],126247/XC941297.ogg,XC,3.5,https://xeno-canto.org/941297,9.0465,-79.3024,Leptodactylus insularum,Spotted Foam-nest Frog,Chris Harrison,cc-by-nc-sa 4.0
20,134933,[''],['advertisement call'],134933/XC941298.ogg,XC,4.0,https://xeno-canto.org/941298,8.6260,-80.1392,Espadarana prosoblepon,Emerald Glass Frog,Chris Harrison,cc-by-nc-sa 4.0
54,21211,[''],[''],21211/XC882646.ogg,XC,4.0,https://xeno-canto.org/882646,NaN,NaN,Allobates femoralis,Spotted-thighed Poison Frog,M.S. Hoogmoed,cc-by 4.0
55,21211,[''],[''],21211/XC882647.ogg,XC,4.0,https://xeno-canto.org/882647,NaN,NaN,Allobates femoralis,Spotted-thighed Poison Frog,M.S. Hoogmoed,cc-by 4.0
56,21211,[''],[''],21211/XC882648.ogg,XC,4.0,https://xeno-canto.org/882648,NaN,NaN,Allobates femoralis,Spotted-thighed Poison Frog,M.S. Hoogmoed,cc-by 4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28534,ywcpar,[''],['flight call'],ywcpar/XC939157.ogg,XC,2.0,https://xeno-canto.org/939157,3.6867,-61.0326,Amazona ochrocephala,Yellow-crowned Parrot,JAYRSON ARAUJO DE OLIVEIRA,cc-by-nc-sa 4.0
28535,ywcpar,[''],['call'],ywcpar/XC939934.ogg,XC,5.0,https://xeno-canto.org/939934,1.7914,-61.1375,Amazona ochrocephala,Yellow-crowned Parrot,JAYRSON ARAUJO DE OLIVEIRA,cc-by-nc-sa 4.0
28536,ywcpar,[''],[''],ywcpar/XC945993.ogg,XC,3.0,https://xeno-canto.org/945993,6.2710,-75.5640,Amazona ochrocephala,Yellow-crowned Parrot,Carlos Delgado JUMABITA,cc-by-nc-sa 4.0
28537,ywcpar,[''],['call'],ywcpar/XC946300.ogg,XC,4.0,https://xeno-canto.org/946300,3.5026,-76.3552,Amazona ochrocephala,Yellow-crowned Parrot,Cristian Rodas,cc-by-nc-sa 4.0


In [54]:
submission.head()

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,soundscape_8358733_5,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,...,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854
1,soundscape_8358733_10,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,...,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854
2,soundscape_8358733_15,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,...,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854,0.004854


In [55]:
# ラベルと生物名の対応
taxonomy.head()

,primary_label,inat_taxon_id,scientific_name,common_name,class_name
0,1139490,1139490,Ragoniella pulchella,Ragoniella pulchella,Insecta
1,1192948,1192948,Oxyprora surinamensis,Oxyprora surinamensis,Insecta
2,1194042,1194042,Copiphora colombiae,Copiphora colombiae,Insecta
3,126247,126247,Leptodactylus insularum,Spotted Foam-nest Frog,Amphibia
4,1346504,1346504,Neoconocephalus brachypterus,Neoconocephalus brachypterus,Insecta


In [56]:
submission = pd.read_csv(os.path.join(cfg.OUTPUT_DIR, "submission.csv"))
submission.head()

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230420_074000_5.0,0.066110,0.084667,0.120852,0.143378,0.147831,0.110816,0.262201,0.081544,0.122699,...,0.082630,0.058091,0.069766,0.066952,0.059076,0.074645,0.056277,0.079561,0.063416,0.057938
1,H02_20230420_074000_10.0,0.208619,0.203479,0.257061,0.279056,0.321855,0.263084,0.418372,0.221314,0.252937,...,0.247504,0.175008,0.217855,0.229229,0.216977,0.239999,0.189182,0.232971,0.193343,0.197549
2,H02_20230420_074000_15.0,0.058337,0.087325,0.124197,0.129605,0.128878,0.114809,0.279394,0.063256,0.103650,...,0.077463,0.044918,0.056708,0.068964,0.058382,0.063521,0.048830,0.059070,0.053869,0.059336
3,H02_20230420_074000_20.0,0.077892,0.069378,0.105745,0.148125,0.147497,0.108865,0.295357,0.073193,0.105156,...,0.084638,0.053950,0.071241,0.067825,0.067954,0.087354,0.060468,0.064882,0.056328,0.064178
4,H02_20230420_074000_25.0,0.066961,0.067450,0.097306,0.122748,0.113538,0.098922,0.275924,0.060624,0.098861,...,0.055698,0.045747,0.057311,0.060764,0.050993,0.062655,0.050726,0.053684,0.047114,0.047656


In [57]:


model_dir_name = "models_20250402_1541"
model_dir = os.path.join(cfg.models_dir, model_dir_name)


# スコアの計算
fold_val_aucs = []
fold_train_aucs = []
fold_best_epochs = []
for fold in range(5):  # fold 0～4 を対象
    log_path = os.path.join(model_dir, f"log_fold{fold}.csv")
    if not os.path.exists(log_path):
        print(f"Missing log for fold {fold}: {log_path}")
        continue

    df = pd.read_csv(log_path)
    best_row = df.loc[df['val_auc'].idxmax()]  # val_aucが最大の行を取得

    best_epoch = int(best_row['epoch'])
    val_auc = best_row['val_auc']
    train_auc = best_row['train_auc']

    print(f"Fold {fold} best epoch: {best_epoch}, val_auc: {val_auc:.3f}, train_auc: {train_auc:.3f}")
    
    fold_val_aucs.append(val_auc)
    fold_train_aucs.append(train_auc)
    fold_best_epochs.append(best_epoch)

# 平均を計算
mean_val_auc = sum(fold_val_aucs) / len(fold_val_aucs)
mean_train_auc = sum(fold_train_aucs) / len(fold_train_aucs)
mean_best_epoch = sum(fold_best_epochs) / len(fold_best_epochs)


# config.csv 読み込み
config_path = os.path.join(model_dir, "config.csv")
config_df = pd.read_csv(config_path)

# 取り出したいキー
important_keys = [
    'model_name','batch_size', 'epochs',
    'optimizer', 'lr', 'weight_decay', 'scheduler', 'min_lr', "tta",
]



# キーと値のペアを辞書として保存
config_dict = {}
# 先にスコアをいれる
config_dict["Note"] = ""
config_dict['Avg Best Epoch'] = f"{mean_best_epoch:.2f}"
config_dict['Avg Train AUC'] = f"{mean_train_auc:.3f}"
config_dict['Avg Val AUC'] = f"{mean_val_auc:.3f}"
config_dict["LB AUC"] = ""


for key in important_keys:
    value = config_df.loc[config_df['key'] == key, 'value'].values
    if len(value) > 0:
        config_dict[key] = value[0]
    else:
        config_dict[key] = ""
        


# スコアも追加



# 出力順
all_keys = list(config_dict.keys())

# マークダウン出力
print("\n```markdown")
# ヘッダー
print("| " + " | ".join(all_keys) + " |")
# 区切り線
print("|" + "|".join(["-" * (len(k)+2) for k in all_keys]) + "|")
# 値
print("| " + " | ".join(str(config_dict[k]) for k in all_keys) + " |")
print("```")


Fold 0 best epoch: 6, val_auc: 0.944, train_auc: 0.974
Fold 1 best epoch: 8, val_auc: 0.950, train_auc: 0.988
Fold 2 best epoch: 7, val_auc: 0.944, train_auc: 0.986
Fold 3 best epoch: 8, val_auc: 0.948, train_auc: 0.990
Fold 4 best epoch: 8, val_auc: 0.948, train_auc: 0.992

```markdown
| Note | Avg Best Epoch | Avg Train AUC | Avg Val AUC | LB AUC | model_name | batch_size | epochs | optimizer | lr | weight_decay | scheduler | min_lr | tta |
|------|----------------|---------------|-------------|--------|------------|------------|--------|-----------|----|--------------|-----------|--------|-----|
|  | 7.40 | 0.986 | 0.947 |  | efficientnet_b0 | 32 | 10 | AdamW | 0.0005 | 1e-05 | CosineAnnealingLR | 1e-06 |  |
```


In [58]:
import torch
import numpy as np

val_auc_list = []
train_auc_list = []
best_epoch_list = []

for i in range(5):
    path = f"../models/models_0401/model_fold{i}.pth"
    checkpoint = torch.load(path, map_location='cpu')
    
    best_epoch = checkpoint.get('epoch') + 1  # epochは0始まりが多いので+1するのが自然
    val_auc = checkpoint.get('val_auc')
    train_auc = checkpoint.get('train_auc')

    print(f"Fold {i}")
    print(f"Best Epoch: {best_epoch}")
    print(f"Val AUC: {val_auc:.3f}")
    print(f"Train AUC: {train_auc:.3f}")
    print()

    val_auc_list.append(val_auc)
    train_auc_list.append(train_auc)
    best_epoch_list.append(best_epoch)

# 平均の計算と表示
mean_val_auc = np.mean(val_auc_list)
mean_train_auc = np.mean(train_auc_list)
mean_best_epoch = np.mean(best_epoch_list)

print("========== Averages ==========")
print(f"Average Best Epoch : {mean_best_epoch:.2f}")
print(f"Average Val AUC    : {mean_val_auc:.3f}")
print(f"Average Train AUC  : {mean_train_auc:.3f}")


Fold 0
Best Epoch: 35
Val AUC: 0.913
Train AUC: 0.976

Fold 1
Best Epoch: 27
Val AUC: 0.924
Train AUC: 0.963

Fold 2
Best Epoch: 31
Val AUC: 0.923
Train AUC: 0.983

Fold 3
Best Epoch: 30
Val AUC: 0.929
Train AUC: 0.983

Fold 4
Best Epoch: 27
Val AUC: 0.926
Train AUC: 0.984

========== Averages ==========
Average Best Epoch : 30.00
Average Val AUC    : 0.923
Average Train AUC  : 0.978


In [73]:
bc20_25_data_dir = os.path.join(cfg.PROCESSED_DIR, "bc20-25")
all_train = pd.read_csv(os.path.join(bc20_25_data_dir, "all_train.csv"))

all_train = all_train[all_train["primary_label"].isin(train_label["primary_label"].unique())]

all_train

,primary_label,secondary_labels,record_name,source,rank
468,ywcpar,[''],XC115515,bc25,1.0
469,ywcpar,[''],XC115516,bc25,2.0
470,ywcpar,[''],XC118809,bc25,3.0
471,ywcpar,[''],XC119486,bc25,4.0
472,ywcpar,[''],XC12092,bc25,5.0
...,...,...,...,...,...
52888,1192948,[''],CSA36366,bc25,2.0
52889,1192948,[''],CSA36373,bc25,3.0
52890,1192948,[''],CSA36388,bc25,4.0
52891,1139490,[''],CSA36385,bc25,1.0


In [81]:
import os
from pathlib import Path
import numpy as np
import soundfile as sf
from tqdm import tqdm

# サンプリングレート
sr = 32000

# 元データと保存先のディレクトリ
bc20_25_data_dir = os.path.join(cfg.PROCESSED_DIR, "bc20-25")
bc20_25_ogg_dir = os.path.join(cfg.PROCESSED_DIR, "bc20-25-ogg")

# 対象のラベルだけ変換
target_labels = [
    "grekis", "greegr", "strher", "linwoo1", "laufal1", "colcha1",
    "blhpar1", "gretin1", "creoro1", "butsal1", "littin1",
    "roahaw", "rutjac1", "smbani", "socfly1"
]

# 変換処理
for primary_label in tqdm(target_labels, desc="Converting to .ogg"):
    src_dir = Path(bc20_25_data_dir) / primary_label
    dst_dir = Path(bc20_25_ogg_dir) / primary_label
    dst_dir.mkdir(parents=True, exist_ok=True)

    # .npy ファイルをすべて取得
    for npy_path in src_dir.glob("*.npy"):
        audio = np.load(npy_path)
        record_name = npy_path.stem  # 拡張子なしファイル名
        ogg_path = dst_dir / f"{record_name}.ogg"
        
        # .ogg 保存
        sf.write(ogg_path, audio, sr, format="OGG", subtype="VORBIS")

Converting to .ogg: 100%|██████████| 15/15 [00:08<00:00,  1.67it/s]


In [75]:
# データを格納するリスト
label_counts = []

# ディレクトリを走査
for label_dir in os.listdir(bc20_25_data_dir):
    full_path = os.path.join(bc20_25_data_dir, label_dir)
    if os.path.isdir(full_path):
        npy_files = [f for f in os.listdir(full_path) if f.endswith('.npy')]
        label_counts.append({
            'primary_label': label_dir,
            'num_files': len(npy_files)
        })

# DataFrameに変換して表示
label_df = pd.DataFrame(label_counts).sort_values('num_files', ascending=False).reset_index(drop=True)
label_df

,primary_label,num_files
0,grekis,86
1,greegr,28
2,strher,20
3,linwoo1,12
4,laufal1,6
5,colcha1,2
6,blhpar1,2
7,gretin1,2
8,creoro1,2
9,butsal1,2


In [80]:
label_df["primary_label"].isin(taxonomy["primary_label"].unique()).sum()

15

In [78]:
label_df["num_files"].sum(), len(all_train) - len(train_label)

(172, 157)

In [74]:
train_label

,primary_label,secondary_labels,type,filename,collection,rating,url,latitude,longitude,scientific_name,common_name,author,license
0,1139490,[''],[''],1139490/CSA36385.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3206,-73.7128,Ragoniella pulchella,Ragoniella pulchella,Fabio A. Sarria-S,cc-by-nc-sa 4.0
1,1139490,[''],[''],1139490/CSA36389.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3206,-73.7128,Ragoniella pulchella,Ragoniella pulchella,Fabio A. Sarria-S,cc-by-nc-sa 4.0
2,1192948,[''],[''],1192948/CSA36358.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3791,-73.7313,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0
3,1192948,[''],[''],1192948/CSA36366.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.2800,-73.8582,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0
4,1192948,[''],[''],1192948/CSA36373.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3791,-73.7313,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28559,ywcpar,[''],[''],ywcpar/iNat77392.ogg,iNat,0.0,https://static.inaturalist.org/sounds/77392.m4a,7.6921,-80.3379,Amazona ochrocephala,Yellow-crowned Parrot,Yennifer Alfaro,cc-by-nc 4.0
28560,ywcpar,[''],[''],ywcpar/iNat78624.ogg,iNat,0.0,https://static.inaturalist.org/sounds/78624.wav,8.9918,-79.4877,Amazona ochrocephala,Yellow-crowned Parrot,Evan Centanni,cc-by-nc-sa 4.0
28561,ywcpar,[''],[''],ywcpar/iNat789234.ogg,iNat,0.0,https://static.inaturalist.org/sounds/789234.wav,9.2316,-70.2041,Amazona ochrocephala,Yellow-crowned Parrot,Henrry,cc-by 4.0
28562,ywcpar,[''],[''],ywcpar/iNat819873.ogg,iNat,0.0,https://static.inaturalist.org/sounds/819873.mp3,10.5838,-66.8545,Amazona ochrocephala,Yellow-crowned Parrot,Alejandro Luy,cc-by-nc 4.0


,primary_label,secondary_labels,type,filename,collection,rating,url,latitude,longitude,scientific_name,common_name,author,license
0,1139490,[''],[''],1139490/CSA36385.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3206,-73.7128,Ragoniella pulchella,Ragoniella pulchella,Fabio A. Sarria-S,cc-by-nc-sa 4.0
1,1139490,[''],[''],1139490/CSA36389.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3206,-73.7128,Ragoniella pulchella,Ragoniella pulchella,Fabio A. Sarria-S,cc-by-nc-sa 4.0
2,1192948,[''],[''],1192948/CSA36358.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3791,-73.7313,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0
3,1192948,[''],[''],1192948/CSA36366.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.2800,-73.8582,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0
4,1192948,[''],[''],1192948/CSA36373.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3791,-73.7313,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28559,ywcpar,[''],[''],ywcpar/iNat77392.ogg,iNat,0.0,https://static.inaturalist.org/sounds/77392.m4a,7.6921,-80.3379,Amazona ochrocephala,Yellow-crowned Parrot,Yennifer Alfaro,cc-by-nc 4.0
28560,ywcpar,[''],[''],ywcpar/iNat78624.ogg,iNat,0.0,https://static.inaturalist.org/sounds/78624.wav,8.9918,-79.4877,Amazona ochrocephala,Yellow-crowned Parrot,Evan Centanni,cc-by-nc-sa 4.0
28561,ywcpar,[''],[''],ywcpar/iNat789234.ogg,iNat,0.0,https://static.inaturalist.org/sounds/789234.wav,9.2316,-70.2041,Amazona ochrocephala,Yellow-crowned Parrot,Henrry,cc-by 4.0
28562,ywcpar,[''],[''],ywcpar/iNat819873.ogg,iNat,0.0,https://static.inaturalist.org/sounds/819873.mp3,10.5838,-66.8545,Amazona ochrocephala,Yellow-crowned Parrot,Alejandro Luy,cc-by-nc 4.0


In [22]:
cfg.models_dir

'../models/'

In [13]:
import timm
print(timm.list_models('*efficientnet*'))

['efficientnet_b0', 'efficientnet_b0_g8_gn', 'efficientnet_b0_g16_evos', 'efficientnet_b0_gn', 'efficientnet_b1', 'efficientnet_b1_pruned', 'efficientnet_b2', 'efficientnet_b2_pruned', 'efficientnet_b3', 'efficientnet_b3_g8_gn', 'efficientnet_b3_gn', 'efficientnet_b3_pruned', 'efficientnet_b4', 'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7', 'efficientnet_b8', 'efficientnet_cc_b0_4e', 'efficientnet_cc_b0_8e', 'efficientnet_cc_b1_8e', 'efficientnet_el', 'efficientnet_el_pruned', 'efficientnet_em', 'efficientnet_es', 'efficientnet_es_pruned', 'efficientnet_l2', 'efficientnet_lite0', 'efficientnet_lite1', 'efficientnet_lite2', 'efficientnet_lite3', 'efficientnet_lite4', 'efficientnetv2_l', 'efficientnetv2_m', 'efficientnetv2_rw_m', 'efficientnetv2_rw_s', 'efficientnetv2_rw_t', 'efficientnetv2_s', 'efficientnetv2_xl', 'gc_efficientnetv2_rw_t', 'tf_efficientnet_b0', 'tf_efficientnet_b1', 'tf_efficientnet_b2', 'tf_efficientnet_b3', 'tf_efficientnet_b4', 'tf_efficientnet_b5', 'tf_eff